In [ ]:
!pip install ortools

In [98]:
import random
from ortools.sat.python import cp_model

def generate_valid_path(size):
    path = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]

    # Randomly select a position for the fixed spot
    fixed_spot = (random.randint(0, size-1), random.randint(0, size-1))

    # Fill the fixed spot with a number between 1 and size*size
    fixed_number = random.randint(1, size * size)

    # Add the fixed spot to the used set
    used = set([fixed_spot])

    # Create a grid and fill it with 0s
    grid = {}
    for i in range(size):
        for j in range(size):
            grid[(i, j)] = 0

    # Set the fixed spot with the selected number
    grid[fixed_spot] = fixed_number

    # Print the initial board state
    print("Initial Board:")
    for i in range(size):
        for j in range(size):
            print(grid[(i, j)], end=" ")
        print()

    if fixed_number == 1:
        path.append(fixed_spot)

    while len(path) < size * size:
        current = path[-1] if path else fixed_spot
        possible_moves = [(current[0] + dx, current[1] + dy) for dx, dy in directions
                          if 0 <= current[0] + dx < size and 0 <= current[1] + dy < size
                          and (current[0] + dx, current[1] + dy) not in used]

        # Exclude the fixed spot's number from possible numbers
        possible_numbers = set(range(1, size * size + 1)) - {fixed_number}

        if not possible_moves:
            path, used = [], set()
        else:
            current = random.choice(possible_moves)
            path.append(current)
            used.add(current)

    # Modify the grid to set the fixed spot's value to 0
    grid[fixed_spot] = 0

    return path, fixed_spot, fixed_number

def solve_rikudo_puzzle(size, path, fixed_spot, fixed_number):
    model = cp_model.CpModel()

    # Create variables for each grid position
    grid = {}
    for i, pos in enumerate(path):
        grid[pos] = model.NewIntVar(1, size * size, f'cell_{pos}')

    # Ensure the sequence is correct
    for i in range(1, len(path)):
        model.Add(grid[path[i]] == grid[path[i-1]] + 1)

    # Set the fixed spot with the fixed number
    grid[fixed_spot] = fixed_number

    # Create a solver and solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        solution_grid = [[0] * size for _ in range(size)]
        for pos, var in grid.items():
            solution_grid[pos[0]][pos[1]] = solver.Value(var)
        return solution_grid
    else:
        return None

# Example usage
size = 6
random_path, fixed_spot, fixed_number = generate_valid_path(size)
solution = solve_rikudo_puzzle(size, random_path, fixed_spot, fixed_number)
if solution:
    print("\nSolution:")
    for row in solution:
        print(' '.join(f"{val:2d}" for val in row))
else:
    print("No solution found.")

Initial Board:
0 0 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 7 0 

Solution:
12 11 10 34 35 36
13  8  9 33 30 29
14 15  7 32 31 28
16  4  5  6 27 25
17  3 20 21 26 24
18 19  2  1  7 23
